In [ ]:
import os
import numpy as np

from Airfoils import airfoil as af

from Visualization import plotting as aplt
from Database.getresults import Database_2D

In [ ]:
HOMEDIR = os.getcwd()

In [ ]:
db = Database_2D(HOMEDIR)
airfoils = db.getAirfoils()
print(f"Airfoils that are computed are: {airfoils}")
airfoil = airfoils[0]
print(f"NACA {airfoil[4:]} polars are:" )
aplt.plotAirfoil(db.Data,airfoil,['All'],size=(10,10))

In [ ]:
reynolds = db.getReynolds(airfoil)
print(f"For NACA {airfoil[4:]} these Reynolds were Computed:\n {reynolds}")
reyn = reynolds[0]
print(f"For Reynolds= {reyn} polars are:")
aplt.plotReynolds(db.Data,airfoil,reyn,['All'],size=(10,10))